In [1]:
%defaultDatasource jdbc:h2:mem:db

# Importando Dados e Criando Tabelas SQL

In [2]:
DROP TABLE IF EXISTS Estado;
DROP TABLE IF EXISTS Cidade;
DROP TABLE IF EXISTS Aeroporto;
DROP TABLE IF EXISTS Periodo;
DROP TABLE IF EXISTS Rota;
DROP TABLE IF EXISTS Voo;
DROP TABLE IF EXISTS Casos;

In [3]:
CREATE TABLE Estado
(
    UF VARCHAR(2) NOT NULL,
    Nome VARCHAR(40) NOT NULL,
    PRIMARY KEY(UF)
)
AS SELECT UF,Nome FROM CSVREAD('https://raw.githubusercontent.com/Desnord/ProjetoFinalMC536/main/stage04/data/processed/estado.csv');

In [4]:
CREATE TABLE Cidade
(
  Estado VARCHAR(2) NOT NULL,
  Nome VARCHAR(40) NOT NULL,
  CONSTRAINT PK_Cidade PRIMARY KEY (Nome,Estado),
  FOREIGN KEY(Estado) REFERENCES Estado(UF)
) 
AS SELECT Estado,Nome FROM CSVREAD('https://raw.githubusercontent.com/Desnord/ProjetoFinalMC536/main/stage04/data/processed/cidadeFINAL.csv');

In [5]:
CREATE TABLE Aeroporto
(
  Sigla VARCHAR(4) NOT NULL,
  Descricao VARCHAR(80) NOT NULL,
  Cidade VARCHAR(40) NOT NULL,
  PRIMARY KEY(Sigla)
) AS SELECT Sigla,Descricao,Cidade FROM CSVREAD('https://raw.githubusercontent.com/Desnord/ProjetoFinalMC536/main/stage04/data/processed/aeroportoFINAL.csv');

In [6]:
CREATE TABLE Periodo
(
  Id INT NOT NULL,
  Semana INT NOT NULL,
  Ano INT NOT NULL,
  PRIMARY KEY(Id,Semana,Ano)
) AS SELECT Id,Semana,Ano FROM CSVREAD('https://raw.githubusercontent.com/Desnord/ProjetoFinalMC536/main/stage04/data/processed/periodo.csv');

In [7]:
CREATE TABLE Rota
(
  Id INT NOT NULL,
  Origem VARCHAR(4) NOT NULL,
  Destino VARCHAR(4) NOT NULL,
  VoosTotais INT NOT NULL,
  PRIMARY KEY(Id),
  FOREIGN KEY(Origem) REFERENCES Aeroporto(Sigla),
  FOREIGN KEY(Destino) REFERENCES Aeroporto(Sigla)
) AS SELECT Id,Origem,Destino,VoosTotais FROM CSVREAD('https://raw.githubusercontent.com/Desnord/ProjetoFinalMC536/main/stage04/data/processed/rota.csv')

In [8]:
CREATE TABLE Voo
(
  Rota INT NOT NULL,
  Periodo INT NOT NULL,
  Quantidade INT NOT NULL,
  CONSTRAINT PK_Rota PRIMARY KEY (Rota,Periodo),
  FOREIGN KEY(Rota) REFERENCES Rota(Id),
  FOREIGN KEY(Periodo) REFERENCES Periodo(Id)
) AS SELECT Rota,Periodo,Quantidade FROM CSVREAD('https://raw.githubusercontent.com/Desnord/ProjetoFinalMC536/main/stage04/data/processed/voo.csv');

In [9]:
CREATE TABLE Casos
(
  Estado VARCHAR(40) NOT NULL,
  Periodo INT NOT NULL,
  NumCasos INT NOT NULL,
  CONSTRAINT PK_Casos PRIMARY KEY (Estado,Periodo),
  FOREIGN KEY(Periodo) REFERENCES Periodo(Id),
  FOREIGN KEY(Estado) REFERENCES Estado(Nome)
) AS SELECT Estado,Periodo,NumCasos FROM CSVREAD('https://raw.githubusercontent.com/Desnord/ProjetoFinalMC536/main/stage04/data/processed/casosFINAL.csv');

# total de casos de gripe por estado, entre 2010 e 2019

In [10]:
select Estado, SUM(NumCasos) TotalDeCasos
from Casos
group by Estado;

# todos os aeroportos de algum estado (no caso SP)

In [11]:
select a.sigla, c.Estado
from Aeroporto a, Cidade c
where a.Cidade = c.Nome and c.Estado = 'SP';

# todas as rotas de algum estado (no caso SP)

In [12]:
select r.Id, a.sigla, c.Estado
from Rota r, Aeroporto a, Cidade c
where a.Cidade = c.Nome and c.Estado = 'SP' and r.Destino = a.Sigla;

# estados do destino de cada rota

In [13]:
select r.Id, r.Destino, c.Estado
from Rota r, Aeroporto a, Cidade c
where r.Destino = a.Sigla and a.Cidade = c.Nome;

# cria duas views para auxiliar nas próximas seleções

In [14]:
drop view RotaCidadeDestino if exists;
drop view EstadoPeriodos if exists;

create view RotaCidadeDestino as 
select r.Id rota, r.Destino destino, c.Estado estado
from Rota r, Aeroporto a, Cidade c
where r.Destino = a.Sigla and a.Cidade = c.Nome;
  
select * from RotaCidadeDestino;

create view EstadoPeriodos as
select e.UF estado,p.Id periodo
from Estado e, Periodo p;

select * from EstadoPeriodos;

# voos e seus estados

In [15]:
select v.Rota,v.Quantidade,v.Periodo, rcd.Estado
from RotaCidadeDestino rcd, Voo v
where rcd.Rota = v.Rota;

# total de voos por periodo, em cada estado (porém faltam os periodos que nao tem nenhum voo)

In [16]:
select rcd.Estado,v.Periodo,count(v.Quantidade)
from RotaCidadeDestino rcd, Voo v
where rcd.Rota = v.Rota
group by rcd.Estado,v.Periodo
order by rcd.Estado,v.Periodo;

# cria outra view auxiliar

In [17]:
drop view VoosEstadoIncompleto if exists;

create view VoosEstadoIncompleto as
    select rcd.Estado estado,v.Periodo periodo,SUM(v.Quantidade) qtd
    from RotaCidadeDestino rcd, Voo v
    where rcd.Rota = v.Rota
    group by rcd.Estado,v.Periodo
    order by rcd.Estado,v.Periodo;

# para todo estado, mostra a quantidade de voos naquele periodo

In [18]:
SELECT ep.estado, ep.periodo, coalesce(vei.qtd, 0) as qtd
FROM EstadoPeriodos as ep
LEFT JOIN VoosEstadoIncompleto as vei
on vei.periodo = ep.periodo and vei.estado = ep.estado;

# adiciona número do casos ao select anterior

In [19]:
drop view EstadoVoosPeriodoCasos if exists;

create view EstadoVoosPeriodoCasos as
    SELECT ep.estado, ep.periodo, coalesce(vei.qtd, 0) as qtd
    FROM EstadoPeriodos as ep
    LEFT JOIN VoosEstadoIncompleto as vei
    on vei.periodo = ep.periodo and vei.estado = ep.estado;

In [20]:
drop view auxiliar if exists;
create view auxiliar as
    select e.UF estado,c.numCasos casos,c.Periodo periodo
    from Casos c, Estado e 
    where e.Nome = c.Estado;

In [25]:
select evpc.estado,evpc.periodo,evpc.qtd voos,aux.casos
from EstadoVoosPeriodoCasos evpc, auxiliar aux
where aux.estado = evpc.estado and evpc.periodo = aux.periodo;

# tranforma periodo em ano e semana

In [29]:
drop view auxiliar2 if exists;

create view auxiliar2 as
select evpc.estado,evpc.periodo,evpc.qtd voos,aux.casos
from EstadoVoosPeriodoCasos evpc, auxiliar aux
where aux.estado = evpc.estado and evpc.periodo = aux.periodo;

In [31]:
select aux.estado,p.semana,p.ano,aux.voos,aux.casos
from Periodo p, auxiliar2 aux
where aux.Periodo = p.Id;